In [1]:
# -*- coding: utf-8 -*-
from miasm.analysis.machine import Machine
from miasm.arch.x86.arch import mn_x86
from miasm.core import parse_asm
from miasm.core.locationdb import LocationDB
from future.utils import viewitems
from miasm.analysis.data_flow import *
import pydotplus
from IPython.display import Image, display_png

In [2]:
loc_db = LocationDB()
asmcfg = parse_asm.parse_txt(mn_x86, 32, ''' 
main:
    PUSH EBP
    MOV EBP, ESP
    MOV ECX, 0x23
    MOV ECX, 0x4
    MOV EAX, ECX
    POP EBP
    RET
''', loc_db)

In [3]:
loc_db.set_location_offset(loc_db.get_name_location("main"), 0x0)

machine = Machine('x86_32')
ir_arch = machine.ira(loc_db)
ircfg = ir_arch.new_ircfg_from_asmcfg(asmcfg)

In [4]:
print('Before Simplification:')
for lbl, irb in viewitems(ircfg.blocks):
    print(irb)

Before Simplification:
b'main':

ESP = ESP + -0x4

@32[ESP + -0x4] = EBP

EBP = ESP

ECX = 0x23

ECX = 0x4

EAX = ECX

ESP = ESP + 0x4

EBP = @32[ESP]

ESP = ESP[0:32] + 0x4

EIP = @32[ESP[0:32]]

IRDst = @32[ESP[0:32]]



In [5]:
deadrm = DeadRemoval(ir_arch)
deadrm(ircfg)

True

In [6]:
print('After Simplification:')
for lbl, irb in viewitems(ircfg.blocks):
    print(irb)

After Simplification:
b'main':

ESP = ESP + -0x4

@32[ESP + -0x4] = EBP



ECX = 0x4

EAX = ECX

ESP = ESP + 0x4

ESP = ESP[0:32] + 0x4

IRDst = @32[ESP[0:32]]

